# CJE Calibration Demo

This notebook demonstrates how to use CJE (Causal Judge Evaluation) to calibrate AI survey responses against human ground truth.

**Key Benefits:**
- Get valid estimates using only 5-10% human labels
- Proper uncertainty quantification with confidence intervals
- Statistical comparison between models/policies

## Installation

```bash
pip install edsl[cje]
```

In [1]:
%pip install -e /Users/eddielandesberg/CIMO/clients/expected-parrot/edsl --force-reinstall --no-deps  

Obtaining file:///Users/eddielandesberg/CIMO/clients/expected-parrot/edsl
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for edsl (pyproject.toml) ... done
  Created wheel for edsl: filename=edsl-1.0.5.dev1-py3-none-any.whl size=6245 sha256=e52322f501395e8eabd710675b421f6dd599a28aa30cd9fcc1da8c19582e583a
  Stored in directory: /private/var/folders/d9/p_hbgpn13k9d30x_f6y1ywn80000gn/T/pip-ephem-wheel-cache-t7ejr1mk/wheels/0b/c3/d9/c17052dd5d7f0c3b74fb530a8cf9b0831dcfa745dc509df62b
Successfully built edsl
  Attempting uninstall: edsl
    Found existing installation: edsl 1.0.5.dev1
    Uninstalling edsl-1.0.5.dev1:
      Successfully uninstalled edsl-1.0.5.dev1

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: /Users/eddielandesberg/.pyenv/versions/3.10.15/bin/python -m pip in

In [2]:
import edsl                                                                                                                                                                               
print(edsl.__file__)    

/Users/eddielandesberg/CIMO/clients/expected-parrot/edsl/edsl/__init__.py


In [3]:
# Install if needed
# %pip install "edsl[cje]"

## Step 1: Run a Survey with Multiple Models

First, we create a simple sentiment survey and run it with two different models.

In [4]:
from edsl import Survey, QuestionLinearScale, Model, Agent

# Create a simple sentiment question
q = QuestionLinearScale(
    question_name="sentiment",
    question_text="Rate the sentiment of this text: '{{ text }}'",
    question_options=[1, 2, 3, 4, 5],
    option_labels={1: "Very Negative", 5: "Very Positive"}
)

survey = Survey([q])

In [5]:
from edsl import ScenarioList

# Sample texts to analyze
texts = [
    "I love this product! It's amazing.",
    "This is okay, nothing special.",
    "Terrible experience, would not recommend.",
    "Pretty good overall, minor issues.",
    "Absolutely fantastic, exceeded expectations!",
    "Meh, it's fine I guess.",
    "Worst purchase ever.",
    "Great quality, fast shipping.",
    "Not worth the money.",
    "Highly recommend to everyone!",
]

scenarios = ScenarioList.from_list("text", texts)

In [6]:
# Run with two models
models = [Model("gpt-4o"), Model("claude-sonnet-4-20250514")]

results = survey.by(scenarios).by(models).run()

In [7]:
# View results
results.select("model", "text", "sentiment")

model.model,scenario.text,answer.sentiment
gpt-4o,I love this product! It's amazing.,5
claude-sonnet-4-20250514,I love this product! It's amazing.,5
gpt-4o,"This is okay, nothing special.",3
claude-sonnet-4-20250514,"This is okay, nothing special.",3
gpt-4o,"Terrible experience, would not recommend.",1
claude-sonnet-4-20250514,"Terrible experience, would not recommend.",1
gpt-4o,"Pretty good overall, minor issues.",4
claude-sonnet-4-20250514,"Pretty good overall, minor issues.",4
gpt-4o,"Absolutely fantastic, exceeded expectations!",5
claude-sonnet-4-20250514,"Absolutely fantastic, exceeded expectations!",5


## Step 2: Add Human Labels (Oracle)

In practice, you would collect human ratings for a sample of responses.
Here we simulate this with "ground truth" ratings for ~50% of samples.

In [8]:
import random

# Simulate human labels for some samples
# In practice, you would collect these from actual human raters
human_labels = []

for i, result in enumerate(results):
    # Add human label for ~50% of samples
    if random.random() < 0.5:
        # Simulate a human rating (1-5 scale)
        text = result["scenario"]["text"]
        if "love" in text or "amazing" in text or "fantastic" in text:
            human_labels.append(5)
        elif "terrible" in text or "worst" in text:
            human_labels.append(1)
        elif "good" in text or "great" in text:
            human_labels.append(4)
        elif "okay" in text or "fine" in text or "meh" in text:
            human_labels.append(3)
        else:
            human_labels.append(3)
    else:
        human_labels.append(None)  # No human label for this sample

print(f"Human labels collected: {sum(1 for x in human_labels if x is not None)}/{len(human_labels)}")

Human labels collected: 11/20


## Step 3: Calibrate with CJE

Now we use CJE to calibrate the AI responses against human labels.

In [9]:
# Transform 1-5 scale to 0-1 for CJE
scale_transform = lambda x: (x - 1) / 4

# Calibrate
cal_result = results.calibrate(
    question_name="sentiment",
    oracle_labels=human_labels,
    policy_column="model",
    score_transform=scale_transform,
    verbose=True,
)

Calibrating with 11/20 oracle labels
Policies: ['gpt-4o', 'claude-sonnet-4-20250514']


Estimator mismatch detected: max|Δ| = 0.2500. This is expected when logged-data and fresh-draws calibrators differ. Using bootstrap's theta_hat for consistency.


In [10]:
# View calibrated estimates
print("Calibrated Estimates:")
print(cal_result)

Calibrated Estimates:
Calibration Results (GOOD)
Oracle samples: 11/20 (55.0%)

Policy Estimates:
  gpt-4o: 0.590 [0.429, 0.785]
  claude-sonnet-4-20250514: 0.340 [0.120, 0.700]


## Step 4: Compare Models

Use statistical testing to compare model performance.

In [11]:
# Get the best performing model
print(f"Best model: {cal_result.best_policy()}")
print(f"Ranking: {cal_result.ranking()}")

Best model: gpt-4o
Ranking: ['gpt-4o', 'claude-sonnet-4-20250514']


In [12]:
# Statistical comparison between models
policies = list(cal_result.estimates.keys())
if len(policies) >= 2:
    comparison = cal_result.compare(policies[0], policies[1])
    print(f"\nComparison: {policies[0]} vs {policies[1]}")
    print(f"Difference: {comparison.difference:.3f}")
    print(f"p-value: {comparison.p_value:.3f}")
    print(f"Significant at α=0.05: {comparison.significant}")


Comparison: claude-sonnet-4-20250514 vs gpt-4o
Difference: -0.250
p-value: 1.000
Significant at α=0.05: False


## Summary

CJE calibration provides:

1. **Efficient use of human labels** - Only need 5-10% oracle labels to calibrate
2. **Valid point estimates** - Corrects for systematic biases in AI judge scores
3. **Uncertainty quantification** - Confidence intervals for each policy
4. **Statistical comparisons** - Proper hypothesis testing between policies

### Next Steps

- Collect real human labels for your survey responses
- Use `CalibrationResult.compare()` to make data-driven model selection decisions
- Monitor for drift by re-calibrating when models are updated